```
--- Day 18: Boiling Boulders ---
You and the elephants finally reach fresh air. You've emerged near the base of a large volcano that seems to be actively erupting! Fortunately, the lava seems to be flowing away from you and toward the ocean.

Bits of lava are still being ejected toward you, so you're sheltering in the cavern exit a little longer. Outside the cave, you can see the lava landing in a pond and hear it loudly hissing as it solidifies.

Depending on the specific compounds in the lava and speed at which it cools, it might be forming obsidian! The cooling rate should be based on the surface area of the lava droplets, so you take a quick scan of a droplet as it flies past you (your puzzle input).

Because of how quickly the lava is moving, the scan isn't very good; its resolution is quite low and, as a result, it approximates the shape of the lava droplet with 1x1x1 cubes on a 3D grid, each given as its x,y,z position.

To approximate the surface area, count the number of sides of each cube that are not immediately connected to another cube. So, if your scan were only two adjacent cubes like 1,1,1 and 2,1,1, each cube would have a single side covered and five sides exposed, a total surface area of 10 sides.

Here's a larger example:

2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5
In the above example, after counting up all the sides that aren't connected to another cube, the total surface area is 64.

What is the surface area of your scanned lava droplet?
```

In [1]:
import numpy as np
from itertools import product
data = np.array([list(map(int,x.split(','))) for x in open('inputs/18.input').read().splitlines()])

min_v, max_v = np.quantile(data,(0,1),axis=0) + [[-1],[1]]

sol_map = np.zeros(max_v - min_v +1 ,dtype=bool)
sol_map[tuple((data-min_v).T)] = True

off_list = list(filter(lambda x: np.abs(x).sum() == 1, product([-1,0,1],repeat=3)))
adjacent_count = [np.logical_and(sol_map, np.roll(sol_map,off,axis=(0,1,2))).sum() for off in off_list]

total_faces = (sol_map*6).sum() - np.sum(adjacent_count)
total_faces

3564

```
--- Part Two ---
Something seems off about your calculation. The cooling rate depends on exterior surface area, but your calculation also included the surface area of air pockets trapped in the lava droplet.

Instead, consider only cube sides that could be reached by the water and steam as the lava droplet tumbles into the pond. The steam will expand to reach as much as possible, completely displacing any air on the outside of the lava droplet but never expanding diagonally.

In the larger example above, exactly one cube of air is trapped within the lava droplet (at 2,2,5), so the exterior surface area of the lava droplet is 58.

What is the exterior surface area of your scanned lava droplet?
```

In [2]:
from itertools import chain
from collections import deque
from typing import Any

search_start = np.array([0,0,0])
visited = set()
q = deque([search_start])
res = 0
while q:
    pos = q.pop()
    if np.any(pos < 0) or np.greater_equal(pos, sol_map.shape).any():
        continue
    pos_tup = tuple(pos)
    if sol_map[pos_tup]:
        res += 1
        continue
    if tuple(pos) in visited:
        continue
    else:
        visited.add(pos_tup)
    q.extend([pos+off for off in off_list])
res

2106